In [1]:
from sail_core.implementation_manager import ImplementationManager
from sail_safe_functions.test.helper_sail_safe_functions.participant_service_local import ParticipantSeriviceLocal
implementation_manager = ImplementationManager.get_instance()
if not implementation_manager.is_initialized:
    implementation_manager.set_participant_service(ParticipantSeriviceLocal())
    implementation_manager.initialize()

In [2]:
from typing import List

import os
import numpy
import pandas

from sail_safe_functions.aggregator.data_frame_federated import DataFrameFederated
from sail_safe_functions.aggregator.data_model.data_model_series import DataModelSeries
from sail_safe_functions.aggregator.series_federated import SeriesFederated
from sail_safe_functions.test.helper_sail_safe_functions.tools_data_test import ToolsDataTest

from sail_safe_functions.aggregator.statistics.estimator import Estimator
from sail_safe_functions.aggregator.statistics.kolmogorov_smirnov_test import KolmogorovSmirnovTest
from sail_safe_functions.aggregator.statistics.kurtosis import Kurtosis
from sail_safe_functions.aggregator.statistics.mean import Mean
from sail_safe_functions.aggregator.statistics.min_max import MinMax

# import tests
from sail_safe_functions.aggregator.statistics.paired_t_test import PairedTTest
from sail_safe_functions.aggregator.statistics.mann_whitney_u_test import MannWhitneyUTest
from sail_safe_functions.aggregator.statistics.wilcoxon_signed_rank_test import WilcoxonSingedRankTest
from sail_safe_functions.aggregator.statistics.spearman import Spearman

from sail_safe_functions.test.helper_sail_safe_functions.generator_one_sample_float import GeneratorOneSampleFloat
from sail_safe_functions.test.helper_sail_safe_functions.generator_two_sample_float import GeneratorTwoSampleFloat
# import helper
from sail_safe_functions.test.helper_sail_safe_functions.tools_privacy import (
    experiment_privacy_one_sample_dma,
)

from sail_safe_functions.test.test_sail_safe_functions.conftest import (
    load_cardio,
    load_gdc_941_1,
    load_house,
    load_investor,
    load_kidney_clean,
)


In [3]:
list_series_number: List[SeriesFederated] = []
#data_path = os.environ["PATH_SAIL_TEST_DATA"]
data_path = "C:\\project\\datascience\\sail-safe-functions\\sail_safe_functions\\test\\data"
list_data_frame = []
list_data_frame.append(load_kidney_clean(data_path))
list_data_frame.append(load_investor(data_path))
list_data_frame.append(load_house(data_path))
list_data_frame.append(load_cardio(data_path))
list_data_frame.append(load_gdc_941_1(data_path))


for data_frame in list_data_frame:
    for series_name in data_frame.list_series_name:
        series = data_frame[series_name]
        if series.data_model_series.type_data_level == DataModelSeries.DataLevelInterval:
            list_series_number.append(series)



In [11]:

from sail_safe_functions.test.helper_sail_safe_functions.estimator_one_sample_reference import EstimatorOneSampleReference
list_estimator = []
list_estimator.append(Mean())
list_estimator.append(MinMax())
list_estimator.append(EstimatorOneSampleReference(MinMax()))

list_extract_fraction = [0.1, 0.2, 0.5, 0.8, 1]
run_count = 2000
print(list_series_number[1].series_name)

list_list_experiment = []

for estimator in list_estimator:
    list_experiment = []

    for extract_fraction in list_extract_fraction:
        experiment = experiment_privacy_one_sample_dma(estimator, list_series_number[1], extract_fraction, run_count)
        list_experiment.append(experiment)

    list_list_experiment.append(list_experiment)


age


c:\project\datascience\sail-safe-functions\sail_safe_functions\aggregator\series.py:12: FutureWarning:

The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.



In [12]:
for list_experiment in list_list_experiment:
    for experiment in list_experiment:
        print(experiment)
from plotly.subplots import make_subplots
import plotly.express as px
import plotly.graph_objects as go

def plot_experiment_privacy_one_sample_dma(list_list_experiment):
    xaxis = "extract_fraction"
    yaxis = "fraction_hit"
    trace_axis = "estimator_name"
    list_split_rows = []
    list_split_cols = []
    list_split_trace = []

    list_subplot_titles = ["boem"]
    fig = make_subplots(rows=1, cols=1, shared_yaxes=True, subplot_titles=list_subplot_titles)
    dict_line = {}
    for i_trace, list_experiment in enumerate(list_list_experiment):
        add_trace_experiment(
            fig,
            1,
            1,
            xaxis,
            yaxis,
            trace_axis,
            True,
            dict_line,
            list_experiment,
        )
    fig["layout"]["xaxis"]["title"] = xaxis
    for i in range(1 - 1):
        fig["layout"][f"xaxis{i+2}"]["title"] =xaxis
    fig["layout"]["yaxis"]["title"] = yaxis
    fig.update_layout(height=350, width=1200, showlegend=True)
    fig.show()



def add_trace_experiment(
    fig,
    index_row: int,
    index_col: int,
    attribute_x: str,
    attribute_y: str,
    attribute_name: str,
    show_legend: bool,
    dict_line: dict,
    list_experiment: List[dict],
):
    print("")
    print(list_experiment)
    name = list_experiment[0]["parameter"][attribute_name]
    # gather data
    list_x = []
    list_y = []
    for experiment in list_experiment:
        list_x.append(experiment["parameter"][attribute_x])
        list_y.append(experiment["result"][attribute_y])
    # sort
    list_x_sorted = [x for x, y in sorted(zip(list_x, list_y), key=lambda pair: pair[0])]
    list_y_sorted = [y for x, y in sorted(zip(list_x, list_y), key=lambda pair: pair[0])]
    # plot
    # go_power = go.Scatter(
    #     x=list_x_sorted,
    #     y=list_y_sorted,
    #     name=name,
    #     line=dict(color="#0000ff"),
    # )
    if name not in dict_line:
        line = dict(color=px.colors.qualitative.Plotly[len(dict_line)])
        dict_line[name] = line

    line = dict_line[name]
    go_power = go.Scatter(
        x=list_x_sorted, y=list_y_sorted, name=name, legendgroup=name, showlegend=show_legend, line=line
    )
    fig.add_trace(go_power, row=index_row, col=index_col)

plot_experiment_privacy_one_sample_dma(list_list_experiment)


{'parameter': {'estimator_name': 'Mean', 'series_name': 'age', 'extract_fraction': 0.1, 'run_count': 2000}, 'result': {'fraction_hit': 0.566}}
{'parameter': {'estimator_name': 'Mean', 'series_name': 'age', 'extract_fraction': 0.2, 'run_count': 2000}, 'result': {'fraction_hit': 0.6095}}
{'parameter': {'estimator_name': 'Mean', 'series_name': 'age', 'extract_fraction': 0.5, 'run_count': 2000}, 'result': {'fraction_hit': 0.6845}}
{'parameter': {'estimator_name': 'Mean', 'series_name': 'age', 'extract_fraction': 0.8, 'run_count': 2000}, 'result': {'fraction_hit': 0.7915}}
{'parameter': {'estimator_name': 'Mean', 'series_name': 'age', 'extract_fraction': 1, 'run_count': 2000}, 'result': {'fraction_hit': 1.0}}
{'parameter': {'estimator_name': 'MinMax', 'series_name': 'age', 'extract_fraction': 0.1, 'run_count': 2000}, 'result': {'fraction_hit': 0.499}}
{'parameter': {'estimator_name': 'MinMax', 'series_name': 'age', 'extract_fraction': 0.2, 'run_count': 2000}, 'result': {'fraction_hit': 0.50